In [1]:
import os
import pandas as pd 
import numpy as np 
import pickle
from sklearn.model_selection import train_test_split

import pickle
import multiprocessing

from time import time

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from gensim.models.phrases import Phrases, Phraser

## Load Data

In [2]:
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

# DOC2VEC
---
## variables for DOC2VEC

In [3]:
maxLen = 150
BATCH_SIZE = 32 #20
MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 200

In [4]:
size = 300
window = 10
epochs = 50

## Bigrams

In [5]:
sent = []
for col in ['question', 'question_body', 'answer']:
    words = [row.split() for row in X_train[col]]
    sent = sent + words

In [6]:
sentences = Phrases(sent)

In [7]:
bigram = Phraser(sentences)

In [8]:
sentences = bigram[sent]

## Gensim word to vector

In [9]:
cores = multiprocessing.cpu_count() 
cores

8

In [10]:
sentences

In [11]:
len(sentences)

30000

In [12]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]

In [13]:
d2v_model = Doc2Vec(min_count=50,
                     window=2,
                     size=300, 
                     alpha=0.025, 
                     min_alpha=0.025)

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [14]:
t = time() 
d2v_model.build_vocab(documents, progress_per=10000)
print('time to build vocabulary : ', format(round((time() - t) / 60, 2)))

time to build vocabulary :  0.13


In [15]:
#t = time() 
#d2v_model.build_vocab(sentences)
#for epoch in range(50):
#    d2v_model.train(sentences)
#    d2v_model.alpha -= 0.002  
#    d2v_model.min_alpha = model.alpha
#print('time to build vocabulary : ', format(round((time() - t) / 60, 2)))

## Training Model

In [16]:
t = time()
d2v_model.train(documents, total_examples=d2v_model.corpus_count, epochs=50, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 2.33 mins


In [17]:
with open('../../data/d2v_pretrained_weights.pickle', 'wb') as handle:
    pickle.dump(d2v_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
pretrained_weights = d2v_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [19]:
d2v_model.most_similar(['python'])

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Python', 0.3155953288078308),
 ('dist_packages', 0.24134019017219543),
 ('appropriate', 0.2262609899044037),
 ('manual', 0.223725825548172),
 ('Unix', 0.2166321575641632),
 ('designed', 0.21517397463321686),
 ('PyQt', 0.2071758657693863),
 ('Perl', 0.19983679056167603),
 ('linux_x', 0.19633597135543823),
 ('recursively', 0.19388741254806519)]

In [20]:
# © Axel CHENU 2021